In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
# Read ygs.txt into a dataframe
#price_of_products_df = pd.read_excel('data_files/ygs.xlsx')
price_of_products_df = pd.read_excel('data_files/ygs_.xlsx')

pricebook_df = pd.read_excel('data_files/pbquery.xlsx')

In [3]:
price_of_products_df.reset_index(inplace=True)

In [4]:
wrong_positions = price_of_products_df[price_of_products_df['Curr.'] != 'EUR']

In [5]:
wrong1 = wrong_positions[wrong_positions['Column1'] == 'EUR']

wrong2 = wrong_positions[wrong_positions['Column1'].isna()]

wrong_temp = wrong_positions[~wrong_positions['Column1'].isna()]

wrong3 = wrong_temp[wrong_temp['Column1'] != 'EUR']

In [6]:
len(wrong_positions) == len(wrong1) + len(wrong2) + len(wrong3)

True

In [ ]:
# fixing first wrong data

# Shift the values in each column by 1
wrong1['Value'] = wrong1['Description_2'] 
wrong1['Description_2'] = wrong1['Del. Time'] 
wrong1['Del. Time'] = wrong1['Min.Order qty']
wrong1['DLP'] = wrong1['Curr.']
wrong1['Curr.'] = wrong1['Column1']
wrong1['Column1'] = ""

wrong1['DLP'] = wrong1['DLP'].astype(str)
wrong1['DLP'] = wrong1['DLP'].str.strip()

In [ ]:
# Create a copy of the DataFrame slice using .copy()
wrong1_copy = wrong1.loc[:, 'Value'].copy()

# Assign values from 'Description_2' column to the copied 'Value' column
wrong1_copy.loc[:] = wrong1.loc[:, 'Description_2']

# Update the original DataFrame with the modified copy
wrong1.loc[:, 'Value'] = wrong1_copy

In [9]:
# fixing wrong data #2 

wrong2 = wrong2.dropna(axis=1)
wrong2[['Description', 'Characteristic', 'Description_1', 'Value', 'Description_2',
       'Del. Time', 'Min.Order qty', 'DLP', 'Curr.', 'Column1', '_3']] = wrong2['Description'].str.split('\t', expand=True)

wrong2['DLP'] = wrong2['DLP'].astype(str)
wrong2['DLP'] = wrong2['DLP'].str.strip()

In [ ]:
# fixing the third case of wrong data

# Split the column value into two columns
wrong3['Description_2'] = wrong3['Curr.']

split_values = wrong3['Curr.'].str.split(' ', 1, expand=True)
wrong3['Value'] = split_values.iloc[:, 0]

wrong3['DLP'] = wrong3['_3']
wrong3['Min.Order qty'] = wrong3['Column1']
wrong3['Curr.'] = 'EUR'

wrong3['DLP'] = wrong3['DLP'].astype(str)
wrong3['DLP'] = wrong3['DLP'].str.strip()

In [11]:
right_positions = price_of_products_df[~(price_of_products_df['Curr.'] != 'EUR')]

In [12]:
final = pd.concat([right_positions, wrong1, wrong2, wrong3])
final_int = final.copy()

In [23]:
final = final_int.copy()

In [15]:
pricebook_df.columns

Index(['Model Number', 'Group', 'Product Hierarchy', 'Model Description',
       'Characteristic', 'Characteristic Characteristic', 'Option',
       'Option Description', 'Price'],
      dtype='object')

In [24]:
final = final.sort_values(by='index')
"""final = final.loc[:, ['Material/Model', 'Product Hierarchy', 'Description',
       'Characteristic', 'Description_1', 'Value', 'Description_2',
       'Del. Time', 'Min.Order qty', 'DLP', 'Curr.']]"""

final = final.rename(columns={'Material/Model': 'Model Number', 
                        'Description': 'Model Description',                          
                        'Description_1': 'Characteristic Description', 
                         'Description_2': 'Option Description',
                        'Value': 'Option',
                        'Del. Time': 'Del Time',
                        'DLP': 'Price'})

# extract the first 5 characters of the 'BU' column
final['Group'] = final['Product Hierarchy'].str.slice(stop=5)


# keep only the desired columns
final = final[['Model Number', 'Group',  'Product Hierarchy', 'Model Description', 'Characteristic', 'Characteristic Description', 'Option', 'Option Description', 'Del Time','Price']]

final['Characteristic'] = final['Characteristic'].astype('str')
# update values
#final['Characteristic'] = final['Characteristic'].apply(lambda x: x.split('_', 1)[1] if '_' in x else x)

final['Option'] = final['Option'].astype('str')
final['Option Description'] = final['Option Description'].astype('str')

# Replace 'nan' values with None in a specific column
final['Characteristic'] = final['Characteristic'].replace('nan', np.nan)
final['Option'] = final['Option'].replace('nan', np.nan)
final['Option Description'] = final['Option Description'].replace('nan', np.nan)

final = final.reset_index()

In [ ]:
# Convert string values to float in a specific column and create a mask for errors
mask = pd.to_numeric(final['Price'], errors='coerce').isna()

# Filter the DataFrame to include only the rows with errors
error_df = final[mask]
normal_df = final[~mask]

normal_df['Price'] = pd.to_numeric(normal_df['Price'], errors='coerce')

error_df['Price'] = error_df['Price'].str.replace('.','').str.replace('-', '').str.replace(',', '.').astype(float)

In [26]:
final = pd.concat([normal_df, error_df])
final = final.sort_values(by='index')
# keep only positions which were in the previous version
models = list(pricebook_df['Model Number'].unique())
final = final[final['Model Number'].isin(models)]
final = final.reset_index(drop=True)

In [37]:
final = final.loc[:, 'Model Number':]
writer = pd.ExcelWriter('data_files/final_for_pb.xlsx')
final.to_excel(writer, sheet_name='pricesheet', index=False)
writer.close()